In [24]:
from glob import glob
import numpy as np


In [25]:
files = glob("*local*")

In [26]:
files

['202109211516_uniform_biased_grid-walk_biased_local_0_seeds.dat',
 '202109211516_uniform_biased_grid-walk_biased_local_0_num_iterations.dat',
 '202109211516_uniform_biased_grid-walk_biased_local_1_samples.dat',
 '202109211516_uniform_biased_grid-walk_biased_local_0_samples.dat',
 '202109211516_uniform_biased_grid-walk_biased_local_1_seeds.dat']

In [27]:
data1, data2 = [np.loadtxt(f) for f in files if "samples" in f]


In [28]:
cov1 = np.cov(data1.T)
print("1")
print(cov1)
cov2 = np.cov(data2.T)
print("2")
print(cov2)


1
[[ 0.03672657 -0.01183858 -0.00956589]
 [-0.01183858  0.02769531 -0.03772006]
 [-0.00956589 -0.03772006  0.16854026]]
2
[[0.03486576 0.01590089 0.01177891]
 [0.01590089 0.02375043 0.02939286]
 [0.01177891 0.02939286 0.18810469]]


In [29]:
print(1)
print(np.mean(data1,axis=0))

print(2)
print(np.mean(data2,axis=0))


1
[0.5267894  0.67225767 0.19402948]
2
[ 0.47222727 -0.72630235  0.05305041]


In [128]:
diff

array([[ 0.44173376, -0.73014897,  0.16416156],
       [ 0.44173376, -0.73014897,  0.16416156],
       [ 0.44173376, -0.73014897,  0.16416156]])

In [6]:
diff = np.broadcast_to(np.mean(data1,axis=0).T,(3,3))
diff
#np.diagonal(diff)*np.identity(3)

array([[0.49429555, 0.70118775, 0.09393761],
       [0.49429555, 0.70118775, 0.09393761],
       [0.49429555, 0.70118775, 0.09393761]])

In [22]:
def online_cov_update(data, burnin):
    b = data[:burnin]
    mean = np.mean(b,axis=0)
    cov = np.cov(b.T)
    rest = data[burnin:]
    t = burnin+1
    for i in range(len(rest)):
        sample = rest[i,:]
        mean = mean + (1/(t))*(sample-mean)
        s = (t/((1+t)*(1+t))) * ((sample-mean).T*(sample-mean))
        diff_mean = np.broadcast_to((sample-mean),(3,3))
        print(diff_mean)
        print( diff_mean.T@(np.diagonal(diff_mean)*np.identity(3)))
        cov = 1/t * (cov*(t-1) + (t/(t-1)) * diff_mean@(np.diagonal(diff_mean)*np.identity(3)))
        #cov = (t/(1+t))*cov
        break
        #cov += s
        t+=1
    print("------")
    print(mean)
    print(cov)
    return cov
            
    

In [23]:
c = online_cov_update(data1,10)

[[0.01010436 0.08694945 0.18552282]
 [0.01010436 0.08694945 0.18552282]
 [0.01010436 0.08694945 0.18552282]]
[[0.0001021  0.00087857 0.00187459]
 [0.00087857 0.00756021 0.01613111]
 [0.00187459 0.01613111 0.03441872]]
------
[ 0.51781264  0.73210355 -0.41014682]
[[0.00686057 0.00193867 0.01405617]
 [0.00119285 0.0015874  0.00624714]
 [0.0106245  0.00356129 0.0218335 ]]


In [15]:
np.mean(data1,0)

array([0.49429555, 0.70118775, 0.09393761])

In [16]:
cov1

array([[ 0.03746178, -0.01174527, -0.0054508 ],
       [-0.01174527,  0.02514399, -0.03226724],
       [-0.0054508 , -0.03226724,  0.19260317]])

In [13]:
c = online_cov_update(data1,10)

[0.49429555 0.70118775 0.09393761]
[[ 0.03745879 -0.01174397 -0.00544909]
 [-0.01174397  0.02514156 -0.0322637 ]
 [-0.00544909 -0.0322637   0.19258591]]


In [53]:
np.linalg.eig(c)


(array([2.44373341e-01, 3.18858168e-05, 3.07863485e-06]),
 array([[-0.57735877, -0.70843629,  0.55527473],
        [-0.57737846, -0.66476807, -0.83149863],
        [-0.57731358,  0.23706842,  0.01673302]]))

In [57]:

np.linalg.eig(cov1)

(array([0.01164295, 0.04246756, 0.190741  ]),
 array([[-0.49485649,  0.86712158, -0.05672063],
        [ 0.84252812,  0.49475358,  0.21299124],
        [-0.21275204, -0.05761137,  0.97540633]]))

In [76]:
class OnlineCovariance:
    """
    A class to calculate the mean and the covariance matrix
    of the incrementally added, n-dimensional data.
    """
    def __init__(self, order):
        """
        Parameters
        ----------
        order: int, The order (=="number of features") of the incrementally added
        dataset and of the resulting covariance matrix.
        """
        self._order = order
        self._shape = (order, order)
        self._identity = np.identity(order)
        self._ones = np.ones(order)
        self._count = 0
        self._mean = np.zeros(order)
        self._cov = np.zeros(self._shape)
        
    @property
    def count(self):
        """
        int, The number of observations that has been added
        to this instance of OnlineCovariance.
        """
        return self._count 
   
    @property
    def mean(self):
        """
        double, The mean of the added data.
        """        
        return self._mean

    @property
    def cov(self):
        """
        array_like, The covariance matrix of the added data.
        """
        return self._cov

    @property
    def corrcoef(self):
        """
        array_like, The normalized covariance matrix of the added data.
        Consists of the Pearson Correlation Coefficients of the data's features.
        """
        if self._count < 1:
            return None
        variances = np.diagonal(self._cov)
        denomiator = np.sqrt(variances[np.newaxis,:] * variances[:,np.newaxis])
        return self._cov / denomiator

    def add(self, observation):
        """
        Add the given observation to this object.
        
        Parameters
        ----------
        observation: array_like, The observation to add.
        """
        if self._order != len(observation):
            raise ValueError(f'Observation to add must be of size {self._order}')
            
        self._count += 1
        delta_at_nMin1 = np.array(observation - self._mean)
        self._mean += delta_at_nMin1 / self._count
        weighted_delta_at_n = np.array(observation - self._mean) / self._count
        import pdb
        pdb.set_trace()
        shp = (self._order, self._order)
        D_at_n = np.broadcast_to(weighted_delta_at_n, self._shape).T
        D = (delta_at_nMin1 * self._identity).dot(D_at_n.T)
        self._cov = self._cov * (self._count - 1) / self._count + D
    
    def merge(self, other):
        """
        Merges the current object and the given other object into a new OnlineCovariance object.

        Parameters
        ----------
        other: OnlineCovariance, The other OnlineCovariance to merge this object with.

        Returns
        -------
        OnlineCovariance
        """
        if other._order != self._order:
            raise ValueError(
                   f'''
                   Cannot merge two OnlineCovariances with different orders.
                   ({self._order} != {other._order})
                   ''')
            
        merged_cov = OnlineCovariance(self._order)
        merged_cov._count = self.count + other.count
        count_corr = (other.count * self.count) / merged_cov._count
        merged_cov._mean = (self.mean/other.count + other.mean/self.count) * count_corr
        flat_mean_diff = self._mean - other._mean
        shp = (self._order, self._order)
        mean_diffs = np.broadcast_to(flat_mean_diff, self._shape).T
        merged_cov._cov = (self._cov * self.count \
                           + other._cov * other._count \
                           + mean_diffs * mean_diffs.T * count_corr) \
                          / merged_cov.count
        return merged_cov

In [77]:
ocov = OnlineCovariance(data1.shape[1])
for observation in data1:
    ocov.add(observation)

> <ipython-input-76-24f75aaa0f0b>(72)add()
     70         import pdb
     71         pdb.set_trace()
---> 72         shp = (self._order, self._order)
     73         D_at_n = np.broadcast_to(weighted_delta_at_n, self._shape).T
     74         D = (delta_at_nMin1 * self._identity).dot(D_at_n.T)



ipdb>  n


> <ipython-input-76-24f75aaa0f0b>(73)add()
     71         pdb.set_trace()
     72         shp = (self._order, self._order)
---> 73         D_at_n = np.broadcast_to(weighted_delta_at_n, self._shape).T
     74         D = (delta_at_nMin1 * self._identity).dot(D_at_n.T)
     75         self._cov = self._cov * (self._count - 1) / self._count + D



ipdb>  n


> <ipython-input-76-24f75aaa0f0b>(74)add()
     72         shp = (self._order, self._order)
     73         D_at_n = np.broadcast_to(weighted_delta_at_n, self._shape).T
---> 74         D = (delta_at_nMin1 * self._identity).dot(D_at_n.T)
     75         self._cov = self._cov * (self._count - 1) / self._count + D
     76 



...

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])


ipdb>  D_at_n.T


array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])


ipdb>  q


BdbQuit: 

In [73]:

ocov.cov

array([[ 0.03538941,  0.01105832, -0.01144641],
       [ 0.01105832,  0.02730784,  0.03632261],
       [-0.01144641,  0.03632261,  0.1821075 ]])

In [74]:
cov1

array([[ 0.03539616,  0.01106043, -0.0114486 ],
       [ 0.01106043,  0.02731306,  0.03632955],
       [-0.0114486 ,  0.03632955,  0.18214228]])

In [75]:
#https://carstenschelp.github.io/2019/05/12/Online_Covariance_Algorithm_002.html